<a href="https://colab.research.google.com/github/matule00/NAEX/blob/main/code/01NAEX_Exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# O1NAEX Exercise 09

author: Matěj Trödler

##	Problem 11.8
from the chapter 11 -  D. C. Montgomery DAoE - 8. edition.

The data  were collected in	an experiment to optimize crystal growth as a function of	three variables $x_1$, $x_2$, and $x_3$. Large values of y (`Yield` in grams)	are desirable. Fit a second-order model and analyze the fitted surface. Under what set of conditions is maximum growth achieved?

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [ ]:
df118 = pd.read_csv("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex118.csv", sep=";")
df118

In [ ]:
model = ols('Yield ~ x1 + x2 + x3 + I(x1**2) + I(x2**2) + I(x3**2) + x1:x2 + x1:x3 + x2:x3', data=df118).fit()
#anova_results = anova_lm(model)
#print(anova_results)

print(model.summary())

All interactions seem to be nonsignificant, therefore we will remove them from the model.

In [ ]:
model_reduced = ols('Yield ~ x2 + x3 + I(x2**2) + I(x3**2)', data=df118).fit()

print(model_reduced.summary())
print(anova_lm(model_reduced))

In [ ]:
anova_compare = anova_lm(model_reduced, model)
print(anova_compare)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define the ranges for x2 and x3
x2_range = np.linspace(df118['x2'].min(), df118['x2'].max(), 100)
x3_range = np.linspace(df118['x3'].min(), df118['x3'].max(), 100)

# Create a grid of x2 and x3 values
x2_grid, x3_grid = np.meshgrid(x2_range, x3_range)

# Fix x1 at a constant value (e.g., the center of its range)
x1_fixed = 0  # Center point for x1

# Prepare a DataFrame for prediction
grid_data = pd.DataFrame({
    'x2': x2_grid.ravel(),
    'x3': x3_grid.ravel(),
    'I(x2**2)': x2_grid.ravel()**2,
    'I(x3**2)': x3_grid.ravel()**2,
})

# Predict the response (Yield) for the grid
y_pred = model_reduced.predict(grid_data).values.reshape(x2_grid.shape)

max_value = y_pred.max()
max_index = np.unravel_index(np.argmax(y_pred, axis=None), y_pred.shape)
max_x2, max_x3 = x2_grid[max_index], x3_grid[max_index]
max_conditions = pd.DataFrame({
    'x2': [max_x2],
    'x3': [max_x3],
    'I(x2**2)': [max_x2**2],
    'I(x3**2)': [max_x3**2]})

prediction = model_reduced.get_prediction(max_conditions)
prediction_summary = prediction.summary_frame(alpha=0.05)  # 95% CI
predicted_max = prediction_summary['mean'].iloc[0]
ci_lower = prediction_summary['mean_ci_lower'].iloc[0]
ci_upper = prediction_summary['mean_ci_upper'].iloc[0]

print('Maximal value prediction 95% CI:\n', prediction_summary.to_string(index=False), '\n')

print("Conditions for Maximum Yield:")
print(max_conditions[['x2', 'x3']].to_string(index=False))


# Create the contour plot
plt.figure(figsize=(10, 6))
contour = plt.contourf(x2_grid, x3_grid, y_pred, levels=20, cmap='viridis')
plt.scatter(max_x2, max_x3, color='green', s=100, marker='o', label=f'Max Yield = {predicted_max:.2f}\n95% CI: ({ci_lower:.2f}, {ci_upper:.2f})')
plt.colorbar(contour, label='Predicted Yield')
plt.scatter(df118['x2'], df118['x3'], color='red', label='Design Points')
plt.xlabel('x2')
plt.ylabel('x3')
plt.title(f'Contour Plot of Yield (x1={x1_fixed})')
plt.legend(loc='best')
plt.show()

In [ ]:
max_observed = df118.loc[df118['Yield'].idxmax()]
max_conditions = pd.DataFrame({
    'x2': [max_observed['x2']],
    'x3': [max_observed['x3']],
    'I(x2**2)': [max_observed['x2']**2],
    'I(x3**2)': [max_observed['x3']**2]
})

prediction = model_reduced.get_prediction(max_conditions)
prediction_summary = prediction.summary_frame(alpha=0.05)  # alpha=0.05 for 95% CI
print("\nPrediction with 95% Confidence Interval:")
print(prediction_summary.to_string(index=False))

##	Problem 11.12
from the chapter 11 -  D. C. Montgomery DAoE - 8. edition.

Consider the three-variable central composite design. Analyze the data and draw conclusions, assuming that we wish to maximize `Conversion` ($y_1$) with			`Activity` ($y_2$) between 55 and 60	achieved?

In [ ]:
df1112 = pd.read_csv("https://raw.githubusercontent.com/francji1/01NAEX/main/data/Ex1112.csv", sep=";")
df1112

Time seems to be nonsignificant in almost all terms - in linear, quadratic and also an interaction with Temperature

In [ ]:
model_conversion = ols('Conversion ~ Time + Temperature + Catalyst + I(Time**2) + I(Temperature**2) + I(Catalyst**2) + Time:Temperature + Time:Catalyst + Temperature:Catalyst', data=df1112).fit()
print(model_conversion.summary())

In [ ]:
model_activity_full = ols('Activity ~ Time + Temperature + Catalyst + I(Time**2) + I(Temperature**2) + I(Catalyst**2) + Time:Temperature + Time:Catalyst + Temperature:Catalyst', data=df1112).fit()
print(model_activity_full.summary())

Since there are many nonsignificant terms, let's try to use the simplier model.

In [ ]:
model_activity = ols('Activity ~ Time + Catalyst + I(Time**2)', data=df1112).fit()
print(model_activity.summary())

In [ ]:
anova_activity = anova_lm(model_activity, model_activity_full)
print(anova_activity)

In [ ]:
time_range = np.linspace(df1112['Time'].min(), df1112['Time'].max(), 100)
catalyst_range = np.linspace(df1112['Catalyst'].min(), df1112['Catalyst'].max(), 100)

time_grid, catalyst_grid = np.meshgrid(time_range, catalyst_range)

temperature_fixed = -1  # Center point for Temperature

grid_data = pd.DataFrame({
    'Time': time_grid.ravel(),
    'Temperature': temperature_fixed,
    'Catalyst': catalyst_grid.ravel(),
    'I(Time**2)': time_grid.ravel()**2,
    'I(Temperature**2)': temperature_fixed**2,
    'I(Catalyst**2)': catalyst_grid.ravel()**2,
    'Time:Temperature': time_grid.ravel() * temperature_fixed,
    'Time:Catalyst': time_grid.ravel() * catalyst_grid.ravel(),
    'Temperature:Catalyst': temperature_fixed * catalyst_grid.ravel()
})

# Predict the response (Conversion) for the grid
y_pred_conversion = model_conversion.predict(grid_data).values.reshape(time_grid.shape)

# Create the contour plot
plt.figure(figsize=(10, 6))
contour = plt.contourf(time_grid, catalyst_grid, y_pred_conversion, levels=20, cmap='viridis')
plt.colorbar(contour, label='Predicted Conversion')
plt.scatter(df1112['Time'], df1112['Catalyst'], color='red', label='Design Points')
plt.xlabel('Time')
plt.ylabel('Catalyst')
plt.title(f'Contour Plot of Conversion (Temperature={temperature_fixed})')
plt.legend(loc='best')
plt.show()

In [ ]:
y_pred_activity = model_activity.predict(grid_data).values.reshape(time_grid.shape)

plt.figure(figsize=(10, 6))
contour = plt.contourf(time_grid, catalyst_grid, y_pred_activity, levels=20, cmap='viridis')
plt.colorbar(contour, label='Predicted Conversion')
plt.scatter(df1112['Time'], df1112['Catalyst'], color='red', label='Design Points')
plt.xlabel('Time')
plt.ylabel('Catalyst')
plt.title(f'Contour Plot of Conversion (Temperature={temperature_fixed})')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))

contour_conversion = plt.contourf(time_grid, catalyst_grid, y_pred_conversion, levels=40, cmap='viridis', alpha=0.6)
plt.clabel(contour_conversion, inline=True, fontsize=8, fmt='%1.1f', colors='blue')
plt.colorbar(contour_conversion, label='Predicted Conversion')

contour_activity = plt.contour(time_grid, catalyst_grid, y_pred_activity, levels=np.linspace(50, 65, 7), colors='black', linewidths=2)
plt.clabel(contour_activity, inline=True, fontsize=8, fmt='%1.1f', colors='black')
activity = Line2D([0], [0], color='black', linewidth=2, label='Predicted Activity')

plt.contourf(time_grid, catalyst_grid, y_pred_activity,
             levels=[55, 60],
             colors='black', alpha=0.15)
red_patch = Patch(color='black', alpha=0.15, label='Activity Range (55-60)')

plt.scatter(df1112['Time'], df1112['Catalyst'], color='red', label='Design Points')
design_points = Line2D([0], [0], color='red', marker='o', linestyle='None', label='Design Points')


activity_mask = (y_pred_activity >= 55) & (y_pred_activity <= 60)
conversion_max_in_activity = np.max(y_pred_conversion[activity_mask])

plt.contourf(time_grid, catalyst_grid, np.where(activity_mask, y_pred_conversion, np.nan),  # Restrict Conversion to Activity mask
             levels=[80, conversion_max_in_activity],  # Small band for visualization
             colors='blue',
             alpha=0.15)
blue_patch = Patch(color='blue', alpha=0.15, label='Maximum on Activity Range')

print('Maximum conversion in activity range:', round(conversion_max_in_activity, 3), '\n')

max_indices = np.where(y_pred_conversion == conversion_max_in_activity)
max_time = time_grid[max_indices][0]  # Time value at max
max_catalyst = catalyst_grid[max_indices][0]  # Catalyst value at max

print(f"Conditions for such maximum:\nTime: {round(max_time ,4)}, \nCatalyst: {round(max_catalyst, 4)}")


#Scatter the maximum on the Activity range
max_indices = np.where(y_pred_conversion == conversion_max_in_activity)
max_time = time_grid[max_indices][0]  # Corresponding 'Time'
max_catalyst = catalyst_grid[max_indices][0]  # Corresponding 'Catalyst'
plt.scatter(max_time, max_catalyst, color='blue', s=100, marker='o', label='Max Conversion on Activity Range')
maximum = Line2D([0], [0], color='blue', marker='o', linestyle='None', label='Max Conversion on Activity Range')

plt.xlabel('Time')
plt.ylabel('Catalyst')
plt.title('Overlay Plot: Conversion and Activity')
plt.legend(loc='best', handles=[blue_patch, red_patch, design_points, maximum, activity])
plt.show()